# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import os
import pandas as pd
import json

In [2]:
response = requests.get('http://api.citybik.es/v2/networks')
print(response.status_code)

200


In [3]:
def get_city_bike_data(city_name):
    url = 'http://api.citybik.es/v2/networks/'
   
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        networks = data.get('networks',[])
        for network in networks:
            if network['location']['city'].lower() == city_name.lower():
                return network
        print(f"'{city_name}' not found in the CityBikes API.")
        return None
    else:
        print(f"Error: Unable to fetch data for {city_name}. Status code: {response.status_code}")
        return None

In [4]:
city_name = 'Vancouver'
city_bike_data = get_city_bike_data(city_name)

if city_bike_data is not None:
    print(json.dumps(city_bike_data, indent=4))

{
    "company": [
        "Vanncouver Bike Share Inc.",
        "CycleHop LLC",
        "City of Vancouver",
        "Shaw Communications Inc.",
        "Fifteen"
    ],
    "gbfs_href": "https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json",
    "href": "/v2/networks/mobibikes",
    "id": "mobibikes",
    "location": {
        "city": "Vancouver",
        "country": "CA",
        "latitude": 49.2827,
        "longitude": -123.1207
    },
    "name": "Mobi"
}


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [5]:
def get_bike_station_details(city_name):
    url = f'http://api.citybik.es/v2/networks/{network_id}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get('network', None)
    else:
        print(f"Error: Unable to fetch data for {network_id}. Status code: {response.status_code}")
        return None
    
network_id = 'mobibikes'
bike_station_details = get_bike_station_details(network_id)
    
if bike_station_details is not None:
    stations = bike_station_details.get('stations', [])
    if not stations:
        print(f"No bike stations data available for {network_id}.")
    else:
        station_details_list = []
        for station in stations:
            station_name = station['name']
            latitude = station['latitude']
            longitude = station['longitude']
            num_bikes = station['free_bikes']
            empty_slots = station['empty_slots']
            station_id = station['id']
            
            station_details_list.append({
                'Station Name': station_name,
                # 'Station ID': station_id,   #didn't want to show it to cluster data...
                'Latitude': latitude,
                'Longitude': longitude,
                'Available Bikes': num_bikes,
                'Empty Slots': empty_slots
            })
        df_city_bike = pd.DataFrame(station_details_list)
else:
    print(f"No data available for {network_id} or network ID not found.")

Put your parsed results into a DataFrame.

In [6]:
df_city_bike

,Station Name,Latitude,Longitude,Available Bikes,Empty Slots
0,10th & Cambie,49.262487,-123.114397,5,30
1,Yaletown-Roundhouse Station,49.274566,-123.121817,15,1
2,Dunsmuir & Beatty,49.279764,-123.110154,4,22
3,12th & Yukon (City Hall),49.260599,-123.113504,4,12
4,8th & Ash,49.264215,-123.117772,5,11
...,...,...,...,...,...
240,Cordova & Granville,49.285670,-123.112543,9,8
241,22nd & Main,49.250940,-123.101306,3,11
242,PNE - Hastings & Windermere,49.280977,-123.035969,2,38
243,1st & Fir,49.270783,-123.141564,9,11


In [7]:
# Export dataframe to csv to use in Foursquare and Yelp
city_bikes = 'city_bikes.csv'
df_city_bike.to_csv(city_bikes, index=False)